In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('Datasets/titanic_train.csv')
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Let's Plan

In [3]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [4]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(['Survived'],axis=1),df['Survived'],test_size=0.2,random_state=42)

In [5]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [8]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [7]:
# imputation transformer 
trfr1 = ColumnTransformer([
    ('Impute_age',SimpleImputer(),[2]),
    ('Impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [13]:
# onehotencoding 
trfr2 = ColumnTransformer([
    ('ohe_sex',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [14]:
trfr3 = ColumnTransformer([
    ('Scaled',MinMaxScaler(),slice(0,10))
])

In [15]:
# Feature Selection 
trfr4 = SelectKBest(score_func=chi2,k=8)

In [16]:
# train the model 
trfr5 = DecisionTreeClassifier()

In [21]:
pipe = Pipeline([
    ('trfr1',trfr1),
    ('trfr2',trfr2),
    ('trfr3',trfr3),
    ('trfr4',trfr4),
    ('trfr5',trfr5)
])

## Pipeline vs make_pipeline

Pipeline requires naming to steps, make_pipelines does not requires
Same Applies to columnstransformers and make_column_transformers

In [19]:
#pipe = make_pipeline(trfr1,trfr2,trfr3,trfr4,trfr5)

In [22]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trfr1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('Impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trfr2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trfr3',
                 ColumnTransformer(transformers=[('Scaled', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trfr4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000000771F19BE50>)),
                ('trfr5', DecisionTreeClassifier())])

In [23]:
from sklearn import set_config
set_config(display='diagram')

In [24]:
pipe.named_steps

{'trfr1': ColumnTransformer(remainder='passthrough',
                   transformers=[('Impute_age', SimpleImputer(), [2]),
                                 ('Impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trfr2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trfr3': ColumnTransformer(transformers=[('Scaled', MinMaxScaler(), slice(0, 10, None))]),
 'trfr4': SelectKBest(k=8, score_func=<function chi2 at 0x000000771F19BE50>),
 'trfr5': DecisionTreeClassifier()}

In [28]:
pipe.named_steps['trfr1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('Impute_age', SimpleImputer(), [2]),
                                ('Impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [29]:
pipe.named_steps['trfr1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('Impute_age', SimpleImputer(), [2]),
                                ('Impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [34]:
pipe.named_steps['trfr1'].transformers_[0][1].statistics_

array([29.49884615])

In [35]:
y_pred = pipe.predict(X_test)

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score(y_test,y_pred)

0.6256983240223464

# Cross_Validation

In [41]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=10,scoring='accuracy')

array([0.61111111, 0.59722222, 0.61971831, 0.63380282, 0.70422535,
       0.66197183, 0.67605634, 0.61971831, 0.61971831, 0.64788732])

## Grid_Search_CV

In [53]:
params={
    'trfr5__max_depth':[1,2,3,4,5,None]
} 
# keep in the mind that trfr5__max_depth parameters take the model name and then double underscore 

In [54]:
from sklearn.model_selection import GridSearchCV

In [55]:
grid = GridSearchCV(pipe,params,cv = 5,scoring='accuracy')

In [56]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trfr1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('Impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('Impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trfr2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trfr3',
                                        ColumnTransformer(transformers=[('Scaled',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trfr4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000000771F19BE50>)),
                                       ('trfr5', DecisionTreeClassifier())]),
             param_grid={'trfr5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [57]:
grid.best_score_

0.6391214419383433

## Exporting the PipeLines 

In [58]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))